In [4]:
import pandas as pd

airports = pd.read_csv(r"C:\Users\htete\Downloads\airports.csv")
weather = pd.read_csv(r"C:\Users\htete\Downloads\weather.csv")

print(airports.head())
print(weather.head())

   faa                           name        lat        lon   alt  tz dst  \
0  04G              Lansdowne Airport  41.130472 -80.619583  1044  -5   A   
1  06A  Moton Field Municipal Airport  32.460572 -85.680028   264  -6   A   
2  06C            Schaumburg Regional  41.989341 -88.101243   801  -6   A   
3  06N                Randall Airport  41.431912 -74.391561   523  -5   A   
4  09J          Jekyll Island Airport  31.074472 -81.427778    11  -5   A   

              tzone  
0  America/New_York  
1   America/Chicago  
2   America/Chicago  
3  America/New_York  
4  America/New_York  
  origin  year  month  day  hour   temp   dewp  humid  wind_dir  wind_speed  \
0    EWR  2013      1    1     1  39.02  26.06  59.37     270.0    10.35702   
1    EWR  2013      1    1     2  39.02  26.96  61.63     250.0     8.05546   
2    EWR  2013      1    1     3  39.02  28.04  64.43     240.0    11.50780   
3    EWR  2013      1    1     4  39.92  28.04  62.21     250.0    12.65858   
4    EWR  

In [5]:
# Northernmost airports
northern_candidates = airports.sort_values('lat', ascending=False).reset_index(drop=True).head(10)
northern_candidates[['faa', 'name', 'lat', 'lon', 'tzone']]


,faa,name,lat,lon,tzone
0,EEN,Dillant Hopkins Airport,72.270833,42.898333,NaN
1,BRW,Wiley Post Will Rogers Mem,71.285446,-156.766003,America/Anchorage
2,AIN,Wainwright Airport,70.638056,-159.994722,America/Anchorage
3,K03,Wainwright As,70.613378,-159.860350,America/Anchorage
4,ATK,Atqasuk Edward Burnell Sr Memorial Airport,70.467300,-157.436000,America/Anchorage
5,UUK,Ugnu-Kuparuk Airport,70.330833,-149.597500,America/Anchorage
6,NUI,Nuiqsut Airport,70.210000,-151.005556,America/Anchorage
7,SCC,Deadhorse,70.194750,-148.465167,America/Anchorage
8,BTI,Barter Island Lrrs,70.133989,-143.581867,America/Anchorage
9,PIZ,Point Lay Lrrs,69.732875,-163.005342,America/Anchorage


In [6]:
# Easternmost airports (two interpretations)

# Method A: raw longitude (largest numeric lon = most east)
east_candidates_raw = airports.sort_values('lon', ascending=False).reset_index(drop=True).head(10)

# Method B: remap longitude to 0–360
airports['lon_mod360'] = (airports['lon'] + 360) % 360
east_candidates_mod360 = airports.sort_values('lon_mod360', ascending=False).reset_index(drop=True).head(10)

print("Top 10 by raw longitude (lon):")
display(east_candidates_raw[['faa','name','lat','lon','tzone']])

print("\nTop 10 by remapped lon_mod360:")
display(east_candidates_mod360[['faa','name','lat','lon','lon_mod360','tzone']])


Top 10 by raw longitude (lon):


,faa,name,lat,lon,tzone
0,SYA,Eareckson As,52.712275,174.113620,America/Anchorage
1,MYF,Montgomery Field,32.475900,117.759000,Asia/Chongqing
2,DVT,Deer Valley Municipal Airport,33.411700,112.457000,Asia/Chongqing
3,EEN,Dillant Hopkins Airport,72.270833,42.898333,NaN
4,EPM,Eastport Municipal Airport,44.910111,-67.012694,America/New_York
5,HUL,Houlton Intl,46.123083,-67.792056,America/New_York
6,CAR,Caribou Muni,46.871500,-68.017917,America/New_York
7,PQI,Northern Maine Rgnl At Presque Isle,46.688958,-68.044797,America/New_York
8,WFK,Northern Aroostook Regional Airport,47.285556,-68.312778,America/New_York
9,BHB,Hancock County - Bar Harbor,44.449769,-68.361565,America/New_York



Top 10 by remapped lon_mod360:


,faa,name,lat,lon,lon_mod360,tzone
0,EPM,Eastport Municipal Airport,44.910111,-67.012694,292.987306,America/New_York
1,HUL,Houlton Intl,46.123083,-67.792056,292.207944,America/New_York
2,CAR,Caribou Muni,46.871500,-68.017917,291.982083,America/New_York
3,PQI,Northern Maine Rgnl At Presque Isle,46.688958,-68.044797,291.955203,America/New_York
4,WFK,Northern Aroostook Regional Airport,47.285556,-68.312778,291.687222,America/New_York
5,BHB,Hancock County - Bar Harbor,44.449769,-68.361565,291.638435,America/New_York
6,ME5,Banks Airport,44.165389,-68.428167,291.571833,America/New_York
7,MLT,Millinocket Muni,45.647836,-68.685561,291.314439,America/New_York
8,BGR,Bangor Intl,44.807444,-68.828139,291.171861,America/New_York
9,RKD,Knox County Regional Airport,44.060111,-69.099230,290.900770,America/New_York


In [7]:
# Windiest NY-area airport on 2013-02-12
w = weather[(weather['year']==2013) & (weather['month']==2) & (weather['day']==12)].copy()

agg = w.groupby('origin').agg(
    max_wind_speed = pd.NamedAgg(column='wind_speed', aggfunc='max'),
    max_wind_gust  = pd.NamedAgg(column='wind_gust', aggfunc='max'),
    mean_wind_speed= pd.NamedAgg(column='wind_speed', aggfunc='mean'),
    count_records  = pd.NamedAgg(column='wind_speed', aggfunc='count'),
).reset_index().sort_values(['max_wind_speed','max_wind_gust'], ascending=False)

agg


,origin,max_wind_speed,max_wind_gust,mean_wind_speed,count_records
0,EWR,1048.36058,31.07106,56.38822,24
2,LGA,23.01560,31.07106,14.96014,24
1,JFK,20.71404,27.61872,14.38475,24
